In [8]:
import torch
from torch import Tensor
from math import ceil
from torch.autograd import Function

DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
import triton
import triton.language as tl

In [29]:
def naive_softmax(x:Tensor) -> Tensor:
    max = x.max()
    x -= x.max(dim = -1)[0][:, None] # this is done for numerical stability
    exp = torch.exp(x)
    return exp / torch.sum(exp, dim=1)[:, None]

# x = torch.rand(100, 200).to(DEVICE)
# triton.testing.assert_close(naive_softmax(x), torch.nn.functional.softmax(x, dim=1))

In [ ]:
@triton.jit
def fused_softmax_kernel(x_pointer, out_pointer, num_elements, blovk_size: tl.constexpr):
    ...

def fused_softmax_triton(x:Tensor, block_size:1024) -> Tensor:
    ...

In [37]:
class FusedSoftmaxFunction(Function):
    @staticmethod
    def forward(ctx, x):
        x = x - x.max(dim = -1)[0][:, None]
        exp = torch.exp(x)
        out = exp / torch.sum(exp, dim=1)[:, None]
        # ctx.save_for_backward()
        return out
    
    @staticmethod
    def backward(ctx, grad_output):
        ...



In [39]:
x = torch.rand(8, 16)
torch.allclose(naive_softmax(x), FusedSoftmaxFunction.apply(x))

True